[Amirhossein Mahmoudi](https://ammahmoudi.github.io)
# Adams–Moulton Method ( with Adams-Bashforth Corrector Support )
solution to [Numerical Analysis (R. Burden, Aires, A. Burden) [10th Ed.]](https://www.amazon.com/Richard-Burden-Numerical-Analysis-Hardcover/dp/B00SB3UL20) Section 5.6 question 6

![5_6_6](images/5_6_6.png)

In [33]:
import pandas as pd
from math import *
import numpy as np

In [34]:
AM_correctors=[
    [2, 1, 1],
    [12, 5, 8, -1],
    [24, 9, 19, -5,1],
    [720, 251, 646, -264, 106,-19]
    
]
AB_correctors = [
	[2, 3, -1],
	[12, 23, -16, 5],
	[24, 55, -59, 37, -9],
	[720, 1901, -2774, 2616, -1274, 251]
]

# adamsMoulton

In [35]:
def adamsMoulton(f, f_actual, t, y, p, h, N,AB_correctors,AM_correctors):
	#coeffients
	AB_corrector = AB_correctors[N-2]
	AM_corrector = AM_correctors[N-2]
	#number of steps
	n = int((p-t)/h)
	#intialize time array
	time = [t+h*i for i in range(n+1)]
	#initialze y_vaules
	y_values = [f_actual(time[i]) for i in range(N)]
	y_values_AB = [f_actual(time[i]) for i in range(N)]
	
	
	#run the algorithm
	for i in range(N, n+1):
		#calculate w_i+1 using AB
		fix = 0
		for j in range(N):
			fix += AB_corrector[j+1] * f(time[i-j-1], y_values[i-j-1])
		yn = y_values[i-1] + h/AB_corrector[0]*fix
		y_values.append(yn)
		y_values_AB.append(yn)
		#calculate w_i+1 using AM
		fix=0
		for j in range(N):
			fix += AM_corrector[j+1] * f(time[i-j], y_values[i-j])
		yn = y_values[i-1] + h/AM_corrector[0]*fix
		y_values[i]=yn

	y_actual=np.ones_like(time)
	for i in range(len(y_values)):
		y_actual[i]=f_actual(time[i])

	return np.c_[time,y_values,y_values_AB,y_actual]


# Test

In [36]:
# f = lambda t, y: (y+1)/t

# f_actual = lambda t: 2*t-1

# t = 1
# y = 1
# p = 1.5
# h = 0.1

f = lambda t, y: 1+y/t+(y/t)**2

f_actual = lambda t: t*tan(log(t))

t = 1
y = 0
p = 3
h = 0.1

result=pd.DataFrame()
for i in range(2,6):
    result_n=adamsMoulton(f, f_actual, t, y, p, h, i,AB_correctors,AM_correctors)
    if i==2:
        result=pd.DataFrame(result_n[:,:3],columns=['time','2-step','2-step(ab)'])
    else:
        result[str(i)+"-step"]=pd.Series(result_n[:,1])
        result[str(i)+"-step-ab"]=pd.Series(result_n[:,2])
result['exact']=pd.Series(result_n[:,3])
result.head(n=12)
    
    

,time,2-step,2-step(ab),3-step,3-step-ab,4-step,4-step-ab,5-step,5-step-ab,exact
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.1,0.105160,0.105160,0.105160,0.105160,0.105160,0.105160,0.105160,0.105160,0.105160
2,1.2,0.221294,0.220871,0.221243,0.221243,0.221243,0.221243,0.221243,0.221243,0.221243
3,1.3,0.349231,0.348819,0.349123,0.349130,0.349121,0.349121,0.349121,0.349121,0.349121
4,1.4,0.489861,0.489430,0.489686,0.489669,0.489682,0.489660,0.489682,0.489682,0.489682
5,1.5,0.644140,0.643671,0.643883,0.643850,0.643876,0.643861,0.643875,0.643882,0.643875
6,1.6,0.813123,0.812599,0.812764,0.812719,0.812753,0.812742,0.812753,0.812756,0.812753
7,1.7,0.997993,0.997402,0.997511,0.997454,0.997495,0.997485,0.997494,0.997495,0.997494
8,1.8,1.200095,1.199422,1.199463,1.199394,1.199440,1.199429,1.199439,1.199439,1.199439
9,1.9,1.420966,1.420196,1.420149,1.420067,1.420118,1.420106,1.420116,1.420115,1.420116


In [37]:
result_3=adamsMoulton(t,p,f_actual,h,f)
df_3=pd.DataFrame(result_3,columns=['time','w','y','iterations'])
df_3.head(20)

,time,w,y,iterations
0,1.0,0.000000,0.000000,0.0
1,1.1,0.105160,0.105160,0.0
2,1.2,0.221243,0.221243,0.0
3,1.3,0.349123,0.349121,7.0
4,1.4,0.489685,0.489682,7.0
5,1.5,0.643880,0.643875,7.0
6,1.6,0.812759,0.812753,7.0
7,1.7,0.997502,0.997494,7.0
8,1.8,1.199449,1.199439,8.0
9,1.9,1.420128,1.420116,8.0
